In [276]:
import numpy as np
import PIL
from PIL import Image
import random
import pandas as pd
import cv2

In [277]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [278]:
def initial_population(IMAGE):
    population = pd.DataFrame(0, index=np.arange(8), columns=['1-R','1-B','1-G','2-R','2-B','2-G','3-R','3-B','3-G','PSNR'])
    for z in population.index:
        im = Image.open(IMAGE)
        A = np.squeeze(im, axis=None)
        E1 = random.choice(A)
        F1 = random.choice(E1)
        E2 = random.choice(A)
        F2 = random.choice(E2)
        E3 = random.choice(A)
        F3 = random.choice(E3)
        Gray1 = F1[0]*0.299 + F1[1]*0.587 + F1[2]*0.114
        Gray2 = F2[0]*0.299 + F2[1]*0.587 + F2[2]*0.114
        Gray3 = F3[0]*0.299 + F3[1]*0.587 + F3[2]*0.114
        B = []
        for i in A:
            for j in i:
                jGray = j[0]*0.299 + j[1]*0.587 + j[2]*0.114
                U = [round(Gray1, 2), round(Gray2, 2), round(Gray3, 2)]
                o = find_nearest(U, jGray)
                if o == round(Gray1, 2):
                    B.append([F1[0],F1[1],F1[2]])
                elif o == round(Gray2, 2):
                    B.append([F2[0],F2[1],F2[2]])
                elif o == round(Gray3, 2):
                    B.append([F3[0],F3[1],F3[2]])
        C = np.array(B)
        D = np.reshape(C,np.shape(A))
        img1 = Image.fromarray(D, 'RGB')
        img1.save('my.tif')
        population['1-R'].iloc[z] = F1[0]
        population['1-B'].iloc[z] = F1[1]
        population['1-G'].iloc[z] = F1[2]
        population['2-R'].iloc[z] = F2[0]
        population['2-B'].iloc[z] = F2[1]
        population['2-G'].iloc[z] = F2[2]
        population['3-R'].iloc[z] = F3[0]
        population['3-B'].iloc[z] = F3[1]
        population['3-G'].iloc[z] = F3[2]
        img_a = cv2.imread(IMAGE)
        img_b = cv2.imread('my.tif')
        PSNR_score = cv2.PSNR(img_a, img_b)
        population['PSNR'].iloc[z] = PSNR_score
        population = population.sort_values(by=['PSNR'], ascending=False)
    return population

In [280]:
def eliminate(population):
    population = population.sort_values(by=['PSNR'], ascending=False)
    population.iloc[2:8,:] = 0
    return population

In [281]:
def cross_over(population):
    Max1_1 = population.iloc[1:2,0:3].values.tolist()
    Max1_2 = population.iloc[0:1,0:3].values.tolist()
    Max1_3 = population.iloc[1:2,3:6].values.tolist()
    Max2_1 = population.iloc[0:1,3:6].values.tolist()
    Max2_2 = population.iloc[1:2,6:9].values.tolist()
    Max2_3 = population.iloc[0:1,6:9].values.tolist()
    Gens = [Max1_1, Max1_2, Max1_3, Max2_1, Max2_2, Max2_3]
    #1
    population.iloc[2:3,0:3] = random.choice(Gens)
    population.iloc[2:3,3:6] = random.choice(Gens)
    population.iloc[2:3,6:9] = random.choice(Gens)
    #2
    population.iloc[3:4,0:3] = random.choice(Gens)
    population.iloc[3:4,3:6] = random.choice(Gens)
    population.iloc[3:4,6:9] = random.choice(Gens)
    #3
    population.iloc[4:5,0:3] = random.choice(Gens)
    population.iloc[4:5,3:6] = random.choice(Gens)
    population.iloc[4:5,6:9] = random.choice(Gens)
    #4
    population.iloc[5:6,0:3] = random.choice(Gens)
    population.iloc[5:6,3:6] = random.choice(Gens)
    population.iloc[5:6,6:9] = random.choice(Gens)
    #5
    population.iloc[6:7,0:3] = random.choice(Gens)
    population.iloc[6:7,3:6] = random.choice(Gens)
    population.iloc[6:7,6:9] = random.choice(Gens)
    #6
    population.iloc[7:8,0:3] = random.choice(Gens)
    population.iloc[7:8,3:6] = random.choice(Gens)
    population.iloc[7:8,6:9] = random.choice(Gens)
    #7
    population.iloc[8:9,0:3] = random.choice(Gens)
    population.iloc[8:9,3:6] = random.choice(Gens)
    population.iloc[8:9,6:9] = random.choice(Gens)
    #8
    population.iloc[2:3,0:3] = random.choice(Gens)
    population.iloc[2:3,3:6] = random.choice(Gens)
    population.iloc[2:3,6:9] = random.choice(Gens)
    return population

In [282]:
def fitness(IMAGE, population):
    
    for z in population.index:
        
        im = Image.open(IMAGE)
        #im.setflags(write=1)
        A = np.squeeze(im, axis=None)
        
        Gray1 = population['1-R'].iloc[z]*0.299 + population['1-B'].iloc[z]*0.587 + population['1-G'].iloc[z]*0.114
        Gray2 = population['2-R'].iloc[z]*0.299 + population['2-B'].iloc[z]*0.587 + population['2-G'].iloc[z]*0.114
        Gray3 = population['3-R'].iloc[z]*0.299 + population['3-B'].iloc[z]*0.587 + population['3-G'].iloc[z]*0.114
        B = []
        U = []
        for i in A:
            for j in i:
                jGray = j[0]*0.299 + j[1]*0.587 + j[2]*0.114
                U = [round(Gray1, 2), round(Gray2, 2), round(Gray3, 2)]
                o = find_nearest(U, jGray)
                if o == round(Gray1, 2):
                    B.append([population['1-R'].iloc[z], population['1-B'].iloc[z], population['1-G'].iloc[z]])
                elif o == round(Gray2, 2):
                    B.append([population['2-R'].iloc[z], population['2-B'].iloc[z], population['2-G'].iloc[z]])
                elif o == round(Gray3, 2):
                    B.append([population['3-R'].iloc[z], population['3-B'].iloc[z], population['3-G'].iloc[z]])
        C = np.array(B)
        D = np.reshape(C,np.shape(A))
        
        img1 = Image.fromarray(D, 'RGB')
        img1.save('my.tif')
        
        img_a = cv2.imread(IMAGE)
        img_b = cv2.imread('my.tif')
        PSNR_score = cv2.PSNR(img_a, img_b)
        population['PSNR'].iloc[z] = PSNR_score
        
    population = population.sort_values(by=['PSNR'], ascending=False)   
    return population

In [283]:
generation = initial_population('Im001_1.tif')
generation

,1-R,1-B,1-G,2-R,2-B,2-G,3-R,3-B,3-G,PSNR
2,161,169,158,139,120,126,67,17,116,28.278851
6,158,164,160,63,16,108,138,118,127,27.947832
5,131,115,118,77,21,110,162,161,157,27.747291
1,161,168,161,73,23,118,153,144,139,26.437601
0,96,76,135,67,30,126,160,166,156,23.882020
4,165,166,161,162,167,163,87,31,120,22.185472
7,80,68,140,163,166,159,160,162,157,22.039804
3,144,124,149,145,129,130,156,164,153,17.927882


In [284]:
generation = eliminate(generation)
generation

,1-R,1-B,1-G,2-R,2-B,2-G,3-R,3-B,3-G,PSNR
2,161,169,158,139,120,126,67,17,116,28.278851
6,158,164,160,63,16,108,138,118,127,27.947832
5,0,0,0,0,0,0,0,0,0,0.000000
1,0,0,0,0,0,0,0,0,0,0.000000
0,0,0,0,0,0,0,0,0,0,0.000000
4,0,0,0,0,0,0,0,0,0,0.000000
7,0,0,0,0,0,0,0,0,0,0.000000
3,0,0,0,0,0,0,0,0,0,0.000000


In [285]:
generation = cross_over(generation)
generation

,1-R,1-B,1-G,2-R,2-B,2-G,3-R,3-B,3-G,PSNR
2,161,169,158,139,120,126,67,17,116,28.278851
6,158,164,160,63,16,108,138,118,127,27.947832
5,67,17,116,138,118,127,161,169,158,0.000000
1,67,17,116,139,120,126,161,169,158,0.000000
0,139,120,126,63,16,108,138,118,127,0.000000
4,158,164,160,67,17,116,67,17,116,0.000000
7,63,16,108,138,118,127,161,169,158,0.000000
3,139,120,126,63,16,108,63,16,108,0.000000


In [287]:
generation

,1-R,1-B,1-G,2-R,2-B,2-G,3-R,3-B,3-G,PSNR
2,161,169,158,139,120,126,67,17,116,28.278851
6,158,164,160,63,16,108,138,118,127,27.947832
5,67,17,116,138,118,127,161,169,158,0.000000
1,67,17,116,139,120,126,161,169,158,0.000000
0,139,120,126,63,16,108,138,118,127,0.000000
4,158,164,160,67,17,116,67,17,116,0.000000
7,63,16,108,138,118,127,161,169,158,0.000000
3,139,120,126,63,16,108,63,16,108,0.000000


In [ ]:
stopping_criterion = 5
generation = initial_population('Im001_1.tif')
j = 1
while j < stopping_criterion:
    generation = eliminate(generation)
    generation = cross_over(generation)
    generation = fitness('Im001_1.tif',generation)
    if j + 1 == stopping_criterion and generation['PSNR'].iloc[0] < 10:
        generation = initial_population('Im001_1.tif')
        j = 1
    j = j + 1
generation